In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from cliffs_delta import cliffs_delta
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

### Configuration of Matplotlib

In [3]:
# –– Nature-ready styling
FONT_SCALE = 1.10  # ← master knob
_BASE_FONT = 12

sns.set_style("whitegrid")
sns.set_context("talk", font_scale=FONT_SCALE)

plt.rcParams.update({
    # 'font.family': 'serif',
    'font.serif': ['Times New Roman', 'Times'],
    'mathtext.fontset': 'stix',
    'axes.linewidth': 1.0,
    'axes.edgecolor': '#000000',    
    'axes.labelsize': _BASE_FONT,
    'axes.titlesize': _BASE_FONT,
    'xtick.major.size': 2,
    'ytick.major.size': 2,
    'xtick.labelsize': _BASE_FONT,
    'ytick.labelsize': _BASE_FONT,
    'legend.fontsize': _BASE_FONT,
    'legend.title_fontsize': _BASE_FONT,
    'figure.dpi': 300,
    'axes.edgecolor': 'grey',
    'axes.linewidth': 1
})

# Fancy boxplot with wider boxes
colors = ['#66c2a5', '#fc8d62']

def plot_boxplot(data, x, y, xlab, ylab, file, rotation=0, dpi=300):
    fig = plt.figure(figsize=(3, 2), constrained_layout=True)
    sns.boxplot(
        x=x,
        y=y,
        data=data,
        # order=median_order,
        hue=x,
        legend=False,
        palette=colors,
        # palette='Set2', # Fancy color palette
        patch_artist=True,
        showfliers=False,
        showcaps=False,
        linewidth=0.8
    )
    plt.yscale('log') # This is the key step

    # plt.title('Delay Time Distribution by Auto Merge Status')
    plt.xlabel(xlab, fontsize=9)
    plt.ylabel(ylab, fontsize=9)
    plt.xticks(rotation=rotation, fontsize=8)
    plt.yticks(fontsize=8)
    plt.grid(axis='y', linestyle='--')
    plt.grid(False)
    plt.savefig(file, bbox_inches='tight', dpi=dpi)
    plt.close(fig)
    # plt.show()

### Load data

In [2]:
prs_df = pd.read_csv('data/clean_prs.csv')

all_upgrades_df = pd.concat(
    (pd.read_csv("data/all_upgrades1.csv"), pd.read_csv("data/all_upgrades2.csv")))

all_upgrades_df.reset_index(drop=True, inplace=True)

all_upgrades_df['last_pr_merged_at'] = pd.to_datetime(
    all_upgrades_df['last_pr_merged_at'], utc=True).dt.tz_convert(None)
all_upgrades_df['last_pr_merged_at'] = pd.to_datetime(
    all_upgrades_df['last_pr_merged_at'], errors='coerce')
all_upgrades_df['pr_created_at'] = pd.to_datetime(
    all_upgrades_df['pr_created_at'], utc=True).dt.tz_convert(None)
all_upgrades_df['pr_created_at'] = pd.to_datetime(
    all_upgrades_df['pr_created_at'], errors='coerce')
all_upgrades_df["commit_label"] = all_upgrades_df["commit_label"].map(
    lambda x: eval(x) if pd.notna(x) else "")

In [3]:
all_upgrades_df.loc[all_upgrades_df['label']=='Merged PR', 'delay_hours'].median()

4.373888888888889

### PR dataset characteristics

In [6]:
def generate_repo_stats_table(df: pd.DataFrame):
    """
    Generate a LaTeX table with repository statistics from GitHub PR data.

    Args:
        df (pd.DataFrame): DataFrame containing GitHub PR data with repository metadata
        output_file (str): Path to output LaTeX file
    """
    # Step 1: Remove duplicate repositories (keep first occurrence)
    df_unique = df.sort_values("stargazer_count", ascending=0).drop_duplicates(
        subset=['repo'], keep='first').copy()

    # Count security PRs per repository
    security_prs = df.groupby('repo').size()
    df_unique['dep_pr_count'] = df_unique['repo'].map(security_prs).fillna(0)

    # Step 3: Prepare the statistics table
    metrics = {
        'Commits': 'repo_commit_count',
        'Age': 'repo_age',
        'Dep. PRs': 'dep_pr_count',
        'PRs': 'pull_request_count',
        'Stars': 'stargazer_count',
        'Contributors': 'mentionable_users_count',
        # 'Issues': 'issue_count',
        # 'Users': 'mentionableUsersCount'
    }

    # Calculate statistics for each metric
    stats_data = []
    for metric_name, col_name in metrics.items():
        values = df_unique[col_name]
        stats_data.append({
            'Metric': metric_name,
            'Min.': np.min(values),
            'Median': np.median(values),
            'Mean': np.mean(values),
            'Max.': np.max(values)
        })

    # Create stats DataFrame
    stats_df = pd.DataFrame(stats_data)
    stats_df['Max.'] = stats_df['Max.'].apply(lambda x: f"{x:,}")
    stats_df['Median'] = stats_df['Median'].apply(lambda x: f"{x:,.2f}")
    stats_df['Mean'] = stats_df['Mean'].apply(lambda x: f"{x:,.2f}")

    # Step 4: Generate LaTeX table
    latex_table = stats_df.to_latex(
        index=False,
        column_format='lrrrr',
        float_format="{:0.2f}".format,
        caption=f'Statistics of the {df['repo'].nunique():,} studied JavaScript projects.',
        label='tab:paper3-repo_stats',
        position='t!'
    )
    latex_table = latex_table.replace(".00", "")
    # Save to file
    # with open(output_file, 'w') as f:
    #     f.write(latex_table)

    return latex_table

In [197]:
print(generate_repo_stats_table(prs_df))

\begin{table}[t!]
\caption{Statistics of the 697 studied JavaScript projects.}
\label{tab:paper3-repo_stats}
\begin{tabular}{lrrrr}
\toprule
Metric & Min. & Median & Mean & Max. \\
\midrule
Commits & 41 & 901 & 2,243.35 & 72,123 \\
Age & 336 & 2,613 & 2,737.24 & 5,910 \\
Dep. PRs & 5 & 47 & 132.81 & 999 \\
PRs & 14 & 373 & 806.99 & 17,558 \\
Stars & 10 & 108 & 622.20 & 10,428 \\
Contributors & 5 & 31 & 104.12 & 7,103 \\
\bottomrule
\end{tabular}
\end{table}



### Repository characteristics

In [8]:
# Step 2: Group by state and count PRs
state_counts = prs_df['state'].value_counts().reset_index()
state_counts.columns = ['state', 'count']

state_counts['state'] = state_counts['state'].str.capitalize()

# Step 3: Calculate percentage
total_dependabot = state_counts['count'].sum()
state_counts['percentage'] = (
    state_counts['count'] / total_dependabot * 100).round(2)
state_counts['percentage'] = state_counts['percentage'].apply(
    lambda x: f"{x:,.2f}\\%")

state_counts['count'] = state_counts['count'].apply(lambda x: f"{x:,}")

# Step 4: Add a total row
total_row = pd.DataFrame({
    'state': ['\\textbf{Total}'],
    'count': ['\\textbf{' + f"{total_dependabot:,}" + '}'],
    'percentage': ['\\textbf{100.00\\%}']
})
state_counts = pd.concat([state_counts, total_row], ignore_index=True)

state_counts.rename(columns={
    'state': '\\textbf{Dependabot security PRs}',
    'count': '\\textbf{\\#}',
    'percentage': '\\textbf{\\%}'
}, inplace=True)

# Step 5: Generate LaTeX table
latex_table = state_counts.to_latex(
    index=False,
    caption='The total count and percentgae of Dependabot security PRs acress different states',
    label='tab:dependabot_prs',
    column_format='lrr',
    position="!hbtp"
)
latex_table = latex_table.replace('\\textbf{Total}', '\\midrule\n\\textbf{Total}')

print(latex_table)

\begin{table}[!hbtp]
\caption{The total count and percentgae of Dependabot security PRs acress different states}
\label{tab:dependabot_prs}
\begin{tabular}{lrr}
\toprule
\textbf{Dependabot security PRs} & \textbf{\#} & \textbf{\%} \\
\midrule
Merged & 55,414 & 59.86\% \\
Closed & 35,142 & 37.96\% \\
Open & 2,011 & 2.17\% \\
\midrule
\textbf{Total} & \textbf{92,567} & \textbf{100.00\%} \\
\bottomrule
\end{tabular}
\end{table}



In [198]:
num_merged_prs = len(prs_df[prs_df['state'] == 'MERGED'])
num_prs_merged_by_dep = len(
    prs_df[(prs_df['state'] == 'MERGED') & (prs_df['merged_by'] == 'dependabot')])
pctg_prs_merged_by_dep = round(
    100 * (num_prs_merged_by_dep / num_merged_prs), 2)
pctg_prs_merged_by_human = 100 - pctg_prs_merged_by_dep

num_closed_prs = len(prs_df[prs_df['state'] == 'CLOSED'])
num_prs_closed_by_dep = len(
    prs_df[(prs_df['state'] == 'CLOSED') & (prs_df['closed_by'] == 'dependabot')])
pctg_prs_closed_by_dep = round(
    100 * (num_prs_closed_by_dep / num_closed_prs), 2)
pctg_prs_closed_by_human = 100 - pctg_prs_closed_by_dep


print(f"The total number of studied Dependabot PRs: {len(prs_df)}")
print(f"The total number of studied repositories: {prs_df['repo'].nunique()}")
print(
    f"The percentage of studied PRs merged by Dependabot: {pctg_prs_merged_by_dep}%")
print(
    f"The percentage of studied PRs merged by others: {pctg_prs_merged_by_human}%")
print(
    f"The percentage of studied PRs closed by Dependabot: {pctg_prs_closed_by_dep}%")
print(
    f"The percentage of studied PRs closed by others: {pctg_prs_closed_by_human}%")

The total number of studied Dependabot PRs: 92567
The total number of studied repositories: 697
The percentage of studied PRs merged by Dependabot: 13.58%
The percentage of studied PRs merged by others: 86.42%
The percentage of studied PRs closed by Dependabot: 83.77%
The percentage of studied PRs closed by others: 16.230000000000004%


### RQ1

#### Findings
 1 & 2

In [10]:
# Count of each label
counts = all_upgrades_df["label"].value_counts()

# Compute exact percentages
percentages = counts / counts.sum() * 100

# Round percentages for display
rounded_percentages = percentages.round(2)

# Adjust the largest percentage to ensure sum is exactly 100%
diff = 100 - rounded_percentages.sum()
if diff != 0:
    # Add/subtract the difference from the largest percentage
    largest_idx = rounded_percentages.idxmax()
    rounded_percentages[largest_idx] += diff

# Median delay per label
median_delay = all_upgrades_df.groupby(
    "label")["delay_time"].median() / (60 * 24)

# Format values
result = pd.DataFrame({
    "Label": counts.index,
    "\\#": counts.apply(lambda x: f"{x:,.0f}"),
    "\\%": rounded_percentages.apply(lambda x: f"{x:.2f}\\%"),
    # median delay
    r"$\tilde{Upg} (days)$": median_delay.apply(lambda x: f"{x:.1f}").reindex(counts.index)
})

# Add total row
total_count = counts.sum()
result.loc[len(result)] = [
    "\\textbf{Total}",
    "\\textbf{" + f"{total_count:,.0f}" + "}",
    "\\textbf{100.00\\%}",
    "-"
]

result.columns = [f"\\textbf{{{col}}}" for col in result.columns]

# Convert to LaTeX
latex_table = result.to_latex(
    index=False,
    escape=False,
    caption="The distribution of Dependabot dependency upgrades.",
    label="tab:paper3-label_dep_upgrade_distribution",
    position="!htbp",   # h=here, t=top, b=bottom, p=page
)

print(latex_table)

\begin{table}[!htbp]
\caption{The distribution of Dependabot dependency upgrades.}
\label{tab:paper3-label_dep_upgrade_distribution}
\begin{tabular}{llll}
\toprule
\textbf{Label} & \textbf{\#} & \textbf{\%} & \textbf{$\tilde{Upg} (days)$} \\
\midrule
Merged PR & 51,713 & 82.62\% & 0.2 \\
Closed PR with external upgrade & 4,661 & 7.45\% & 4.6 \\
Superseding merged PR & 4,616 & 7.37\% & 22.6 \\
Superseding closed PR with external upgrade & 1,601 & 2.56\% & 43.5 \\
\textbf{Total} & \textbf{62,591} & \textbf{100.00\%} & - \\
\bottomrule
\end{tabular}
\end{table}



In [11]:
median_upgrade_time = all_upgrades_df['delay_hours'].median()
v1 = all_upgrades_df.loc[all_upgrades_df['label']
                         == 'Merged PR', 'delay_hours'].median()
v2 = all_upgrades_df.loc[all_upgrades_df['label']
                         != 'Merged PR', 'delay_hours'].median()
print(f"The overall median upgrade time is {median_upgrade_time:.2f} hours ({(median_upgrade_time/24):.2f} days).")
print(f"Our defined categories show a an upgrade {v2/v1:.2f} times than merged PRs")
print(f"The median upgrade time of Merged PRs is {v1:.2f} hours.")
print(f"The median upgrade time of non-Merged PRs is {v2:.2f} hours ({v2/24:.2f} days).")

The overall median upgrade time is 8.03 hours (0.33 days).
Our defined categories show a an upgrade 80.40 times than merged PRs
The median upgrade time of Merged PRs is 4.37 hours.
The median upgrade time of non-Merged PRs is 351.65 hours (14.65 days).


#### Plot the overall upgrade time

In [27]:
def plot_overall_upgrade_time(data: pd.DataFrame, log_scale: bool = True, filename=None):
    labels = ['Merged PR', 'Other upgrades']

    # Create figure
    fig, ax = plt.subplots(figsize=(5.5, 3.3))
    
    sns.boxplot(
        data=data, x="label", y="delay_hours",
        # labels=labels,
        # notch=True,
        showfliers=True,
        showcaps=False, #
        patch_artist=True,
        hue='label',
        palette=colors,
        saturation=0.8, linewidth=1.2, ax=ax,
        medianprops={"linewidth": 1, "color": "#000000"},
    )
        
    ax.set_xlabel('Upgrade type')
    ax.set_ylabel('Upgrade time')

    if log_scale:
        ax.set_yscale("symlog")
    ax.set_ylim(-0.1)
    hours_yticks = [0, 1, 2, 8, 24 * 2, 24 * 14, 24 * 90, 24 * 360, 24 * 1440]
    ax.set_yticks(hours_yticks)
    hours_ylabels = []
    for h in hours_yticks:
        if h < 24:
            hours_ylabels.append(f"{h}h")
        elif h < 24 * 30:
            hours_ylabels.append(f"{h // 24}d")
        else:
            hours_ylabels.append(f"{h // (24 * 30)}mo")
    ax.set_yticklabels(hours_ylabels)
    
    # Annotate counts inside boxes
    labels_unique = data['label'].unique()
    # In Seaborn 0.12+, each box is the first patch for the category in order
    for i, label in enumerate(labels_unique):
        n = len(data[data['label'] == label])
        box_patch = ax.patches[i]  # the first patch per box
        bbox = box_patch.get_path().get_extents()  # get bounding box from PathPatch
        y_top = bbox.y1
        y_bottom = bbox.y0
        y_text = y_bottom + (y_top - y_bottom) * 0.80  # slightly below top
        ax.text(i, y_text, f"{n:,}", ha='center', va='top', fontsize=12, color='black')

    handles, labels = ax.get_legend_handles_labels()
    uniq = [(h, l) for i, (h, l) in enumerate(zip(handles, labels))
            if l not in labels[:i]]

    # Only attempt to create a legend if 'uniq' is not empty
    if uniq:
        ax.legend(*zip(*uniq), frameon=False, ncol=2, loc="upper right")

    # Replace underscores with spaces / nicer names
    # ax.set_xticklabels([NAME_MAPPING.get(lbl.get_text(), lbl.get_text())
                        # for lbl in ax.get_xticklabels()])
    ax.grid(False)

    ax.set_ylim(top=40*(data['delay_hours'].max()//24)) 
    
    fig.tight_layout()
    # out_fp = Path(out_fp)
    # out_fp.parent.mkdir(parents=True, exist_ok=True)
    if filename:
        fig.savefig(filename, bbox_inches='tight', pad_inches=0.01, dpi=300)
        plt.close(fig)
    else:
        plt.show()

In [18]:
# Compute delay in days
all_upgrades_df['delay_hours'] = all_upgrades_df['delay_time'] / (60)

# Masks
merged_mask = all_upgrades_df['label'] == 'Merged PR'
delay_mask = all_upgrades_df['delay_time'] > 0

# Extract delays with labels
merged_df = all_upgrades_df.loc[merged_mask & delay_mask, ['delay_hours']].copy()
merged_df['label'] = 'Merged PRs'

other_df = all_upgrades_df.loc[~merged_mask & delay_mask, ['delay_hours']].copy()
other_df['label'] = 'Other upgrades'

# Concatenate into a single long-format DataFrame
delay_df = pd.concat([merged_df, other_df], ignore_index=True)

In [28]:
plot_overall_upgrade_time(delay_df, filename='figures/RQ1/overall_upgrade_time.pdf')

##### Ths size of of the chain of superseded PRs

In [29]:
all_upgrades_df.loc[all_upgrades_df['label'].isin(['Superseding merged PR', 'Superseding closed PR with external upgrade']), 'continuous_superseded_count'].describe()

count    6217.000000
mean        2.607849
std         4.504037
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max       154.000000
Name: continuous_superseded_count, dtype: float64

#### Finding 3

#### To select random sample

In [ ]:
random_sample_df = (all_upgrades_df[all_upgrades_df['label'].isin(['Superseding closed PR with external upgrade', 'Closed PR with external upgrade'])]
 .sample(n=362, random_state=41))
random_sample_df['url'] = "https://www.github.com/" + random_sample_df['repo'].str.cat(random_sample_df['matching_commit_hash'], "/commit/")
random_sample_df = random_sample_df[['url', 'id', 'title', 'label']]
random_sample_df['label'] = None

#### Stats of the manual analysis

In [83]:
def rename_label(x):
    if x in ['lock-family', 'lock:family']:
        return 'groups:family'
    elif x == 'release:prep':
        return 'release-prep'
    elif x in ['group-dev', 'group:dev']:
        return 'groups:dev'
    elif x in ['group-prod', 'group:prod']:
        return 'groups:prod'
    else:
        return x
    
def adjust_labels(x):
    if x == 'groups':
        return "Grouping dependencies"
    elif x == 'others':
        return "Others"
    elif x == 'bot':
        return "Upgraded by other bots"
    elif x == 'release-prep':
        return "Upgraded in the new release"
    elif x == 'different-version':
        return "Upgraded to a newer version"
    elif x == 'required-source-code-changes':
        return "Required source code changes"
    elif x == 'core-lib-major-upgrade':
        return "Upgraded along a core library"
    elif x == 'bug fixes':
        return "Fix bugs"
    elif x == 'change-lockfile':
        return "Create lockfile"
    elif x == 'operator':
        return "Change operator"
    
def generate_rq1_manual_analysis_latex_table(df):
    
    df['high_label'] = df['high_label'].map(adjust_labels)
    
    # Compute counts
    counts = df["high_label"].value_counts(dropna=False)

    # Compute percentages
    perc = df["high_label"].value_counts(normalize=True, dropna=False) * 100

    # Round and correct to sum to 100%
    perc_rounded = perc.round(2)
    difference = 100 - perc_rounded.sum()
    perc_rounded.iloc[-1] += difference

    # Build table with high_label included as a column
    table = pd.DataFrame({
        "Change type": counts.index,
        "\\#": counts.values,
        "\\%": perc_rounded.values
    })

    # Optional: sort if needed
    table = table.sort_values("\\#", ascending=False)
    
    latex_table = table.to_latex(
        index=False,   # important: keeps high_label as a column
        float_format="%.2f",
        caption="Different changes that developers make outside Dependabot PRs.",
        label="tab:high_label_dist"
    )
    
    return latex_table

def extract_repo(x):
    res = x.split("/")
    return res[-4] + '/' + res[-3]

In [84]:
random_sample_df = pd.read_csv("data/rq1_manual_analysis.csv")

random_sample_df.iloc[:, 3] = random_sample_df.iloc[:, 3].map(rename_label)
random_sample_df['high_label'] = random_sample_df['label'].map(lambda x: x.split(':')[0])

### RQ2

#### Finding 1: Clustering project based on their upgrade time into four groups

In [30]:
# 1. Compute the median delay_time per project
project_median_delay = all_upgrades_df.groupby('repo')['delay_time'].median().reset_index(name='median_delay_time')
# project_median_delay['median_delay_time'] /= 60

# 2. Define time thresholds in MINUTES
# 1 hour = 60 minutes
HOUR_THRESHOLD = 60
# 1 day = 24 hours * 60 minutes/hour = 1440 minutes
DAY_THRESHOLD = 1440
# 1 week = 7 days * 1440 minutes/day = 10080 minutes
WEEK_THRESHOLD = 10080

# 2. Define the bins, starting at 0 and including the max value
min_delay = project_median_delay['median_delay_time'].min()
max_delay = project_median_delay['median_delay_time'].max()

bins = [
    0,
    HOUR_THRESHOLD,
    DAY_THRESHOLD,
    WEEK_THRESHOLD
]

# Ensure the maximum delay time is included by adding it as the last bin edge
if max_delay > WEEK_THRESHOLD:
    bins.append(max_delay + 0.001)

# Remove duplicates and sort to ensure monotonic bins
bins = sorted(list(set(bins)))

labels = [
    '0) Within 1 Hour',
    '1) Within 1 Day',
    '2) Within 1 Week',
    '3) Beyond 1 Week'
]

# 3. Create the 'delay_cluster' column
project_median_delay['delay_cluster'] = pd.cut(
    project_median_delay['median_delay_time'],
    bins=bins,
    labels=labels[:len(bins)-1],
    right=True, # (Exclusive start, inclusive end)
    include_lowest=True
)

# 4. Count the results
cluster_counts = project_median_delay['delay_cluster'].value_counts().reindex(labels, fill_value=0)

# 2. Create the summary table (DataFrame)
total_projects = cluster_counts.sum()
count_and_percent_df = pd.DataFrame({
    'Count': cluster_counts,
    'Percentage': (cluster_counts / total_projects) * 100
})

# 3. Format the percentage column for a clean output
count_and_percent_df['Percentage'] = count_and_percent_df['Percentage'].map('{:.2f}%'.format)#.reset_index(drop=-1)

print(count_and_percent_df.to_latex())

\begin{tabular}{lrl}
\toprule
 & Count & Percentage \\
delay_cluster &  &  \\
\midrule
0) Within 1 Hour & 70 & 10.04% \\
1) Within 1 Day & 189 & 27.12% \\
2) Within 1 Week & 187 & 26.83% \\
3) Beyond 1 Week & 251 & 36.01% \\
\bottomrule
\end{tabular}



In [34]:
project_median_delay['delay_cluster'].value_counts()

delay_cluster
3) Beyond 1 Week    251
1) Within 1 Day     189
2) Within 1 Week    187
0) Within 1 Hour     70
Name: count, dtype: int64

##### How many (percentage) projects have at least 20\% of the upgrades whose time to upgrade is higher than 1 week?

In [35]:
# 1. Identify upgrades that took more than a week (7 days)
all_upgrades_df['is_late'] = all_upgrades_df['delay_days'] > 7

# 2. Group by repo to calculate the percentage of late upgrades per project
# We calculate the mean of the boolean 'is_late' which gives the fraction (0.0 to 1.0)
repo_stats = all_upgrades_df.groupby('repo')['is_late'].mean().reset_index()

# 3. Count how many projects have at least 20% (0.20) of their upgrades late
late_projects_count = len(repo_stats[repo_stats['is_late'] >= 0.20])

# 4. Calculate the percentage of such projects out of the total
total_projects = len(repo_stats)
percentage = (late_projects_count / total_projects) * 100

print(f"Percentage of projects with >= 20% late upgrades: {percentage:.2f}%")

Percentage of projects with >= 20% late upgrades: 68.58%


#### Finding 2

In [41]:
def plot_barplot(data, filename=None):
    # --- Custom binning ---
    def custom_bin(p):
        if p == 0:
            return "0%"
        elif 1 <= p <= 10:
            return "1-10%"
        else:
            lower = int(p // 10 * 10)
            upper = lower + 10
            if lower == 100:
                lower, upper = 90, 100
            return f"{lower}-{upper}%"

    binned_labels = data.apply(custom_bin)

    # Correct ordered categories
    ordered_bins = [
        "0%", "1-10%", "10-20%", "30-40%", "40-50%", 
        "50-60%", "60-70%", "70-80%", "80-90%", "90-100%",
    ]

    percentage_counts = binned_labels.value_counts()

    # Prepare data for seaborn
    plot_df = pd.DataFrame({
        "range": percentage_counts.index,
        "count": percentage_counts.values
    })

    # Convert range to ordered categorical and sort
    plot_df["range"] = pd.Categorical(plot_df["range"], categories=ordered_bins, ordered=True)
    plot_df = plot_df.sort_values("range")

    # Plot
    plt.figure(figsize=(5.5, 3.3))
    ax = sns.barplot(data=plot_df, x="range", y="count", edgecolor="none", color='grey')

    plt.xlabel("% range of upgrades made externally")
    plt.ylabel("# of projects")
    
    # --- The Fix: Set horizontal alignment to 'right' ---
    plt.xticks(rotation=45, ha='right', rotation_mode='anchor')

    plt.tight_layout()
    
    if filename:
        plt.savefig(filename, bbox_inches='tight', pad_inches=0.01, dpi=300)
        plt.close()
    else:
        plt.show()

In [39]:
# Define the target labels
target_labels = [
    "Closed PR with external upgrade",
    "Superseding closed PR with external upgrade"
]

# Compute the percentage for each repo
repo_percentages = (
    all_upgrades_df.assign(is_target=all_upgrades_df['label'].isin(target_labels))
    .groupby('repo')['is_target']
    .mean() * 100  # percentage
)

In [42]:
plot_barplot(
    data=repo_percentages,
    filename='figures/RQ2/pctg_external_upgrades.pdf'
)

In [43]:
(repo_percentages > 0).sum()/len(repo_percentages)

0.7862266857962698

### RQ3

#### Preprocessing

In [ ]:
dependabot_changes_df = pd.read_csv('data/dependabot_changes.csv')
dependabot_changes_df['date'] = pd.to_datetime(dependabot_changes_df['date'], utc=True).dt.tz_convert(None)

In [ ]:
dependabot_changes_df['repo'].nunique(),697, len(dependabot_changes_df)

In [ ]:
changed_per_repo_count = dependabot_changes_df.groupby('repo').size().reset_index(name='changes_count')
changed_per_repo_count['changes_count'].describe()

In [ ]:
def interpret_effect_size(delta):
    abs_delta = abs(delta)
    if abs_delta < 0.147:
        return "negligible"
    elif abs_delta < 0.33:
        return "small"
    elif abs_delta < 0.474:
        return "medium"
    else:
        return "large"
    
    
def compute_cliffs_for_dependabot_changes(dependabot_changed_df, all_upgrades_df):
    # ensure datetime
    dependabot_changed_df = dependabot_changed_df.copy()
    all_upgrades_df = all_upgrades_df.copy()

    dependabot_changed_df = dependabot_changed_df.sort_values(['repo', 'date']).reset_index(drop=True)
    results = []
    counter = 0
    for repo, group in dependabot_changed_df.groupby('repo'):
        # sort upgrades
        repo_upgrades = (
            all_upgrades_df[all_upgrades_df['repo'] == repo]
            .sort_values('pr_created_at')
        )

        changes = group.reset_index(drop=True)
        for i, row in changes.iterrows():
            change_date = row['date']

            # Determine the start of the BEFORE window
            if i == 0:
                # No earlier change -> before window starts at -inf
                previous_change_date = pd.Timestamp.min
            else:
                previous_change_date = changes.iloc[i - 1]['date']

            # Determine the end of the AFTER window
            next_date = changes.iloc[i + 1]['date'] if i < len(changes) - 1 else pd.Timestamp.max

            # New requirement:
            # BEFORE = upgrades between (previous_change_date, change_date)
            before_mask = (
                (repo_upgrades['pr_created_at'] > previous_change_date) &
                (repo_upgrades['pr_created_at'] < change_date)
            )

            # AFTER stays the same
            after_mask = (
                (repo_upgrades['pr_created_at'] >= change_date) &
                (repo_upgrades['pr_created_at'] < next_date)
            )

            before_times = repo_upgrades.loc[before_mask, 'delay_days'].dropna().values
            after_times = repo_upgrades.loc[after_mask, 'delay_days'].dropna().values

            # Default outputs
            delta, size, median_before, median_after = np.nan, np.nan, np.nan, np.nan

            # Compute metrics only if both groups exist
            if len(before_times) > 0 and len(after_times) > 0:
                delta_tuple = cliffs_delta(before_times, after_times)
                delta = delta_tuple[0]
                size = interpret_effect_size(delta)
                median_before = np.median(before_times)
                median_after = np.median(after_times)

            results.append({
                'repo': repo,
                'date': change_date,
                'cliffs_delta': delta,
                'effect_size': size,
                'median_before': median_before,
                'median_after': median_after,
                'num_upgrade_before': before_mask.sum(), 
                'num_upgrade_after': after_mask.sum() 
            })
        counter += len(changes)

    metrics_df = pd.DataFrame(results)
    result_out = dependabot_changed_df.merge(metrics_df, on=['repo', 'date'], how='left')
    result_out =result_out.drop_duplicates(subset=['repo', 'commit_hash'], keep='last').reset_index(drop=True)

    return result_out

In [ ]:
result_df = compute_cliffs_for_dependabot_changes(dependabot_changes_df, all_upgrades_df)

no_upgrade_pctg = 100 * (result_df['median_before'].isna().sum()/len(result_df))
print(f"The percentages of changes that had no upgrade before and after: {no_upgrade_pctg:.2f}, upgrades with median before and after: {result_df['median_before'].notna().sum()}")
result_df.loc[result_df['median_after'].notna(), "num_upgrade_after"].describe()

In [ ]:
100 - no_upgrade_pctg

In [ ]:
result_df = result_df[result_df['median_before'].notna()]

result_count_df = result_df.groupby('repo').size().reset_index(name="change_count")

clean_repos = result_count_df.loc[result_count_df['change_count']>=1, 'repo'].unique()

result_df = result_df[result_df['repo'].isin(clean_repos)]

result_df['commit_url'] = result_df['repo'].str.cat(result_df['commit_hash'], '/commit/')

In [ ]:
def summarize_dependabot_effectiveness(result_df):
    result_df = result_df.rename(columns={"effect_size": "Effect size"})
    summary = (
        result_df
        .groupby('Effect size')
        .apply(lambda x: pd.Series({
            'Fast': (x['cliffs_delta'] > 0).sum(),
            'Slow': (x['cliffs_delta'] < 0).sum(),
            'Total': len(x),
            'median_delta': x['cliffs_delta'].median(),
            'median_before': x['median_before'].median(),
            'median_after': x['median_after'].median(),
            'median_ratio_before_after': (x['median_before'].median() / x['median_after'].median())
            if x['median_after'].median() != 0 else np.nan
        }), include_groups=False)
        .sort_values('Fast')
        .reset_index()
    )
    for col in summary.columns[1:4]:
        summary[col] = summary[col].astype(int)
    return summary

In [ ]:
summary_df = summarize_dependabot_effectiveness(result_df.copy())
latex_table = summary_df.iloc[:, :4].to_latex(
    index=False,
    escape=False,
    column_format="lccc",  # adjust as needed
    label="tab:paper3-dependabot-actions",
    caption="The frequency of developer changes to Dependabot config file that contribute to fast and slow upgrades.",
    position="h!",
)
print(latex_table)

In [ ]:
len(result_df.loc[result_df['effect_size']=='large']),len(result_df['repo'])

In [ ]:
result_df.loc[result_df['effect_size']=='large', 'repo'].nunique(),result_df['repo'].nunique()

#### Report stats of Dependabot configuration changes associated with fast and slow upgrades

In [ ]:
fast_actions_df = pd.read_excel("./data/clean_acc_actions_final.xlsx")
slow_actions_df = pd.read_excel("./data/clean_slow_actions_final.xlsx")

In [ ]:
DEPENDABOT_OPTION_MAPPING = {
    'Adopt Dependabot': {
        'high-option': 'Config file'
    },
    'Remove Dependabot': {
        'high-option': 'Config file'
    },
    'Rename file': {
        'high-option': 'Config file'
    },
    'Other ecosystem': {
        'label': 'Others',
        'high-option': 'Others'
    },
    'Others': {
        'high-option': 'Others'
    },
    'Enable production dependency updates': {
        'label': 'Others',
        'high-option': 'Others'
    },
    'Remove allow': {
        'label': 'Others',
        'high-option': 'Others'
    },
    'Many changes': {
        'label': 'Others',
        'high-option': 'Others'
    },
    'Increase interval (e.g., from daily to weekly)': {
        'high-option': 'schedule'
    },
    'Decrease interval': {
        'high-option': 'schedule'
    },
    'Set time': {
        'label': 'Add/set time',
        'high-option': 'schedule'
    },
    'Add/set time': {
        'high-option': 'schedule'
    },
    'Add/Set time': {
        'label': 'Add/set time',
        'high-option': 'schedule'
    },
    'Increase limit (e.g., from 5 to 20)': {
        'high-option': 'open-pull-request-limit'
    },
    'Decrease limit': {
        'high-option': 'open-pull-request-limit'
    },
    'Fix typos': {
        'high-option': 'Style'
    },
    'Ignore unstable dependencies/version': {
        'high-option': 'ignore'
    },
    'Unignore dependencies': {
        'high-option': 'ignore'
    },
    'Ignore a dependency that does not support version updates': {
        'high-option': 'ignore'
    },
    'Ignore major version upgrades': {
        'high-option': 'ignore'
    },
    'Ignore dependency updates': {
        'high-option': 'ignore'
    },
    'Ignoring peer-dependency conflicts': {
        'label': 'Ignore peer-dependency conflicts',
        'high-option': 'ignore'
    },
    'Ignore minor/patch version upgrades': {
        'high-option': 'ignore'
    },
    'Ignore ESM-related dependencies': {
        'high-option': 'ignore'
    },
    'Add npm ecosystem': {
        'label': 'Add/remove updates for npm',
        'high-option': 'package-ecoystem'
    },
    'Remove npm ecosystem': {
        'label': 'Add/remove updates for npm',
        'high-option': 'package-ecoystem'
    },
    'Group dev/prod dependencies': {
        'high-option': 'groups'
    },
    'Group matching name dependencies': {
        'high-option': 'groups'
    },
    'Group all dependencies': {
        'high-option': 'groups'
    },
    'Ungroup dev dependencies': {
        'high-option': 'groups'
    },
    'Add commit-message': {
        'label': 'Add/update/remove commit-message',
        'high-option': 'commit-message'
    },
    'Update commit-message': {
        'label': 'Add/update/remove commit-message',
        'high-option': 'commit-message'
    },
    'Add labels': {
        'label': 'Add/update/remove labels',
        'high-option': 'labels'
    },
    'Remove labels': {
        'label': 'Add/update/remove labels',
        'high-option': 'labels'
    },
    'Update labels': {
        'label': 'Add/update/remove labels',
        'high-option': 'labels'
    },
    'Change reviewer': {
        'label': 'Add/update/remove reviewers',
        'high-option': 'reviewers'
    },
    'Modify reviewers': {
        'label': 'Add/update/remove reviewers',
        'high-option': 'reviewers'
    },
    'Remove reviewers': {
        'label': 'Add/update/remove reviewers',
        'high-option': 'reviewers'
    },
    'Reconfigure reviewers': {
        'label': 'Add/update/remove reviewers',
        'high-option': 'reviewers'
    },
    'Add reviewers': {
        'label': 'Add/update/remove reviewers',
        'high-option': 'reviewers'
    },
    'Update reviewer': {
        'label': 'Add/update/remove reviewers',
        'high-option': 'reviewers'
    },
    'Add target-branch': {
        'label': 'Add/update/remove target-branch',
        'high-option': 'target-branch'
    },
    'Remove target-branch': {
        'label': 'Add/update/remove target-branch',
        'high-option': 'target-branch'
    },
    'Rename target-branch': {
        'label': 'Add/update/remove target-branch',
        'high-option': 'target-branch'
    },
    'Modify target-branch': {
        'label': 'Add/update/remove target-branch',
        'high-option': 'target-branch'
    },
    'Rebase strategy': {
        'label': 'Add rebase-strategy',
        'high-option': 'rebase-strategy'
    },
    'Increase versioning-strategy': {
        'label': 'Increase/widen versioning-strategy',
        'high-option': 'versioning-strategy'
    },
    'Change to increase-if-necessary': {
        'label': 'Increase/widen versioning-strategy',
        'high-option': 'versioning-strategy'
    },
    'Increase-if-necessary': {
        'label': 'Increase/widen versioning-strategy',
        'high-option': 'versioning-strategy'
    },
    'Increase': {
        'label': 'Increase/widen versioning-strategy',
        'high-option': 'versioning-strategy'
    },
    'Widen': {
        'label': 'Increase/widen versioning-strategy',
        'high-option': 'versioning-strategy'
    },
    'Add assignees': {
        'label': 'Add/update assignees',
        'high-option': 'assignees'
    },
    'Modify assignees': {
        'label': 'Add/update assignees',
        'high-option': 'assignees'
    },
}

In [ ]:
def map_label(label, mapping):
    """Return (option, change_label)"""
    if label in mapping:
        option = mapping[label].get("high-option", label)
        change = mapping[label].get("label", label)
    else:
        option = label
        change = label
    return option, change


def count_by_option_and_change(df, mapping):
    counts = defaultdict(lambda: defaultdict(int))
    
    for label in df["label"]:
        option, change = map_label(label, mapping)
        counts[option][change] += 1
    
    return counts



In [ ]:
# Count fast and slow occurrences
fast_counts = count_by_option_and_change(fast_actions_df, DEPENDABOT_OPTION_MAPPING)
slow_counts = count_by_option_and_change(slow_actions_df, DEPENDABOT_OPTION_MAPPING)

# Collect all options and changes
options = sorted(set(fast_counts) | set(slow_counts))

rows = []
others = None
for option in options:
    changes = sorted(set(fast_counts[option]) | set(slow_counts[option]))
    for change in changes:
        item = {
            "Option": option,
            "Change": change,
            "Fast": fast_counts[option].get(change, 0),
            "Slow": slow_counts[option].get(change, 0),
        }
        if option == 'Others':
            others = item.copy()
        else:
            rows.append(item)
rows.append(others)

table_df = pd.DataFrame(rows)

table_df["Total"] = table_df["Fast"] + table_df["Slow"]

option_totals = (
    table_df[table_df["Option"] != "Others"]
    .groupby("Option")["Total"]
    .sum()
    .sort_values(ascending=False)
)

ordered_options = list(option_totals.index)

if "Others" in table_df["Option"].values:
    ordered_options.append("Others")
    
table_df["Option"] = pd.Categorical(
    table_df["Option"],
    categories=ordered_options,
    ordered=True
)

table_df = table_df.sort_values(
    ["Option", "Total"],
    ascending=[True, False]
)

table_df = table_df.drop(columns="Total")

summary_df = (
    table_df
    .groupby("Option", as_index=False)[["Fast", "Slow"]]
    .sum()
)

# Compute total for sorting
summary_df["Total"] = summary_df["Fast"] + summary_df["Slow"]

# Sort all except Others
sorted_options = (
    summary_df[summary_df["Option"] != "Others"]
    .sort_values("Total", ascending=False)["Option"]
    .tolist()
)

if "Others" in summary_df["Option"].values:
    sorted_options.append("Others")

summary_df["Option"] = pd.Categorical(
    summary_df["Option"],
    categories=sorted_options,
    ordered=True
)

summary_df = (
    summary_df
    .sort_values("Option")
    .drop(columns="Total")
)

In [ ]:
def summary_to_latex(df):
    lines = []
    lines.append(r"\begin{table*}[t!]")
    lines.append(r"\centering")
    lines.append(r"\footnotesize")
    lines.append(r"\caption{Comparison of fast and slow actions that are made to the Dependabot configuration file.}\label{tab:paper3-slow_fast_actions_count}")
    lines.append(r"\begin{tabular}{>{\arraybackslash}p{3cm} r r}")
    lines.append(r"\toprule")
    lines.append(r"\textbf{Action} & \textbf{Fast (\#)} & \textbf{Slow (\#)} \\")
    lines.append(r"\midrule")

    for _, row in df.iterrows():
        lines.append(
            rf"{row['Option']} & {row['Fast']} & {row['Slow']} \\"
        )

    lines.append(r"\bottomrule")
    lines.append(r"\end{tabular}")
    lines.append(r"\end{table*}")

    return "\n".join(lines)

def to_latex_with_rowspan(df):
    latex = []
    latex.append(r"\begin{table*}[t!]")
    latex.append(r"\centering")
    latex.append(r"\footnotesize")
    latex.append(r"\caption{Distribution of Dependabot changed options that are associated with fast and slow upgrades.}")
    latex.append(r"\label{tab:paper3-manual-analysis-dependabot-config}")
    latex.append(r"\begin{tabularx}{\linewidth}{>{\arraybackslash}p{2.6cm} X r r}")
    latex.append(r"\toprule")
    latex.append(r"\textbf{Option} & \textbf{Change} & \textbf{Fast (\#)} & \textbf{Slow (\#)} \\")
    latex.append(r"\midrule")

    for option, group in df.groupby("Option", sort=False):
        rowspan = len(group)
        first = True
        for _, row in group.iterrows():
            if first:
                latex.append(
                    rf"\multirow{{{rowspan}}}{{*}}{{\centering {option}}} & "
                    rf"{row['Change']} & {row['Fast']} & {row['Slow']} \\"
                )
                first = False
            else:
                latex.append(
                    rf" & {row['Change']} & {row['Fast']} & {row['Slow']} \\"
                )
        latex.append(r"\hline")

    latex.append(r"\end{tabularx}")
    latex.append(r"\end{table*}")

    return "\n".join(latex)

In [ ]:
latex_summary_table = summary_to_latex(summary_df)
print(latex_summary_table)

In [ ]:
latex_table = to_latex_with_rowspan(table_df)
print(latex_table)

### RQ4

#### Finding 1: Upgraded across types of dependencies

##### Plot the boxplot

In [104]:
def plot_topicgpt_boxplot(data: pd.DataFrame, log_scale=True, filename=None):
    topicgpt_freq_df = data.dropna(subset=['topicgpt_label']).groupby('topicgpt_label').size().reset_index(name='count')

    TOP_X_LABELS = topicgpt_freq_df.sort_values('count', ascending=False).iloc[:, 0].values.tolist()

    # 2. Filter the DataFrame
    df_filtered = data[data['topicgpt_label'].isin(TOP_X_LABELS)].copy()

    # 3. Sort the labels by the median delay_time for visualization clarity
    median_order = df_filtered.groupby('topicgpt_label')['delay_time'].median().sort_values(ascending=False).index

    # Calculate counts for annotation
    counts = df_filtered['topicgpt_label'].value_counts().loc[median_order]

    # Calculate the overall median of the filtered data
    overall_median = df_filtered['delay_hours'].median()

    # 4. Generate the boxplot with modifications
    fig, ax, = plt.subplots(figsize=(10, 4.5))

    # Use a 'fancy' color palette (e.g., 'Spectral')
    sns.boxplot(
        x='topicgpt_label',
        y='delay_hours',
        data=df_filtered,
        order=median_order,
        boxprops=dict(facecolor=(0,0,0,0)),
        # palette='Set2', # Fancy color palette
        patch_artist=True,
        showfliers=False,
        showcaps=False,
        linewidth=0.8,
        # boxprops=dict(alpha=0.6, edgecolor='black', facecolor='white')
    )

    # Apply log scale to the y-axis (Delay Time)
    plt.yscale('log')

    # Add a horizontal line for the overall median
    # Remember to use the log-transformed value for 'y' if you use axhline on a log-scaled plot
    # The 'delay_time' values are already in the 'days' unit
    plt.axhline(
        y=overall_median,
        color='r',            # Red line
        linestyle='--',       # Dashed line
        linewidth=1.5,
        label=f'Overall median: {overall_median:.2f} hours'
    )

    if log_scale:
        ax.set_yscale("symlog")
    ax.set_ylim(-0.1)
    hours_yticks = [0, 1, 2, 8, 24 * 1, 24 * 4, 24 * 14, 24 * 60, 24 * 180, 24 * 360]
    ax.set_yticks(hours_yticks)
    hours_ylabels = []
    for h in hours_yticks:
        if h < 24:
            hours_ylabels.append(f"{h}h")
        elif h < 24 * 30:
            hours_ylabels.append(f"{h // 24}d")
        else:
            hours_ylabels.append(f"{h // (24 * 30)}mo")
    ax.set_yticklabels(hours_ylabels)

    plt.xlabel('Type of dependency')
    plt.ylabel('Upgrade time') # Updated unit
    # plt.ylabel([f"{b:g}" for b in original_breaks])

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45, ha='right')


    # Add annotations for the number of instances (counts)
    for i, label in enumerate(median_order):
        count = counts.loc[label]
        
        # Place the annotation above the 90th percentile of the data for that group, adjusted for log scale
        y_pos = df_filtered[df_filtered['topicgpt_label'] == label]['delay_hours'].quantile(0.80) * 1.65
        
        # Fallback for placement
        if pd.isna(y_pos) or y_pos < df_filtered['delay_hours'].min():
            y_pos = df_filtered['delay_hours'].median() * 1.65

        plt.text(
            x=i,
            y=y_pos,
            s=f"{count:,}",
            # color='black',
            ha='center',
            fontsize=8,
            weight='bold'
        )


    # Adjust layout to prevent labels from being cut off
    plt.tight_layout()
    plt.grid(False)

    if filename:
        # Save the figure
        plt.savefig(filename, bbox_inches='tight', pad_inches=0.01, dpi=300)
        plt.close(fig)
    else:    
        plt.show()

In [107]:
plot_topicgpt_boxplot(
    all_upgrades_df, 
    filename='figures/RQ4/upgrade_time_per_dep_type.pdf'
)

In [108]:
all_upgrades_df.loc[(all_upgrades_df['lib_repo']=='babel/babel'), 'delay_hours'].describe()

count     4396.000000
mean       158.701175
std        816.804490
min          0.000556
25%          0.069653
50%          2.440556
75%         31.364028
max      15831.001111
Name: delay_hours, dtype: float64

In [86]:
topicgpt_data_df = pd.read_json("./data/output/sample/assignment_corrected.jsonl", lines=True)
topicgpt_data_df.rename(columns={'id': 'lib_repo', 'label': 'topicgpt_label'}, inplace=True)

In [ ]:
all_upgrades_with_topics_df = all_upgrades_df[all_upgrades_df['lib_repo'].isin(topicgpt_data_df['lib_repo'].values)].copy()
# all_upgrades_with_topics_df['delay_time'] = all_upgrades_with_topics_df['delay_time'] / (60 * 24)
all_upgrades_with_topics_df = pd.merge(
    all_upgrades_with_topics_df,
    prs_df[["repo_pr_id", "auto_merge_allowed"]],
    on="repo_pr_id",
    how="left"
)

In [93]:
all_upgrades_df['topicgpt_label'].value_counts()

topicgpt_label
Linting                10360
Utilities               8666
Compiler                6017
Testing Tools           4792
Type definitions        4665
UI Libraries            4510
Web Services            3887
Bundler                 3287
Parser                  2788
Web Frameworks          1684
CLI Libraries           1455
Plugins                 1245
Security Tools          1208
Versioning              1009
Filesystem Tools         964
Documentation            930
Logging Tools            893
WebSocket Libraries      863
i8n Libraries            799
Data Validation          446
Configuration Tools      408
Database Tools           398
Package management       378
Location Services        258
Data Serialization       228
Name: count, dtype: int64

In [94]:
all_upgrades_df.loc[all_upgrades_df['topicgpt_label']=='Logging', 'delay_hours'].median()

nan

In [88]:
all_upgrades_df.dropna(subset=["topicgpt_label"]).groupby(['topicgpt_label'])['delay_hours'].median().reset_index(name="median_upgrade_time").sort_values('median_upgrade_time')

,topicgpt_label,median_upgrade_time
5,Data Validation,1.064167
17,Type definitions,1.650833
12,Package management,3.057500
2,Compiler,3.966389
20,Versioning,4.902222
14,Plugins,5.605556
9,Linting,5.928611
18,UI Libraries,7.138750
11,Logging Tools,7.419167
16,Testing Tools,7.634722


In [91]:
all_upgrades_df.loc[all_upgrades_df['topicgpt_label']=='Data Serialization', 'delay_hours'].describe()


count      228.000000
mean       553.879136
std       1540.387147
min          0.006944
25%          2.185486
50%         40.636389
75%        466.309097
max      18917.949722
Name: delay_hours, dtype: float64

##### Statical analysis between dev and prod dependencies

In [90]:
v1 = all_upgrades_df.loc[all_upgrades_df['dep_type'].notna()&(all_upgrades_df['dep_type']=="prod"), "delay_hours"]
v2 = all_upgrades_df.loc[all_upgrades_df['dep_type'].notna()&(all_upgrades_df['dep_type']=="dev"), "delay_hours"]
v1.median(), v2.median(), cliffs_delta(v1, v2)

(14.807777777777778, 6.254722222222222, (0.12947093522567513, 'negligible'))

##### Draw the latex table

In [388]:
unique_topicgpt_data_df = topicgpt_data_df.drop_duplicates(subset=['topicgpt_label'])[['topicgpt_label', 'responses', 'lib_repo']]
unique_topicgpt_data_df.rename(columns={"topicgpt_label": "Label", "responses": "Definition", "lib_repo": "Library"}, inplace=True)

In [390]:
# Convert to LaTeX with vertical lines
latex_code = unique_topicgpt_data_df.to_latex(
    index=False, 
    escape=False, 
    column_format="|l|r|r|",  # vertical lines between columns
    header=True
)


# Add horizontal lines manually
latex_code = latex_code.replace("\\toprule", "\\hline")
latex_code = latex_code.replace("\\midrule", "\\hline")
latex_code = latex_code.replace("\\bottomrule", "\\hline")

# Add horizontal lines after each row
lines = latex_code.splitlines()
new_lines = []
for line in lines:
    if "&" in line and "\\" in line and not line.strip().startswith("\\hline"):
        line = line.rstrip("\\") + " \\\\ \\hline"  # add \hline after each row
    new_lines.append(line)

latex_code = "\n".join(new_lines)

latex_code = f"\\begin{{adjustbox}}{{width=\\columnwidth}}\n{latex_code}\n\\end{{adjustbox}}"

# Wrap everything in table environment with adjustbox
full_table = f"""
\\begin{{table}}[t!]
\\centering
{latex_code}
\\caption{{Example Table Caption}}
\\label{{tab:example}}
\\end{{table}}
"""

print(full_table)


\begin{table}[t!]
\centering
\begin{adjustbox}{width=\columnwidth}
\begin{tabular}{|l|r|r|}
\hline
Label & Definition & Library  \\ \hline
\hline
Utilities & [1] Utilities: The description describes a cross-platform wrapper around Node's child-process APIs (spawn/spawnSync), which matches Utilities' scope of general-purpose helpers and cross-platform abstractions. (Supporting quote: "A cross platform solution to node's spawn and spawnSync") & moxystudio/node-cross-spawn  \\ \hline
HTTP & [1] HTTP: The description describes an HTTP server that serves static files and implements HTTP features like Range requests and conditional GET handling, which are specific to the HTTP protocol (e.g., handling request/response headers and RFC behaviors). (Supporting quote: "Streaming static file server with Range and conditional-GET support") & pillarjs/send  \\ \hline
Parser & [1] Parser: This is a library whose primary purpose is parsing semantic-version strings for Node (i.e., converting version t

### RQ5

#### Finding 1: Upgrade time across the change types

In [139]:
labeled_commits_upgrades_df = all_upgrades_df.dropna(subset=['commit_label']).copy()
labeled_commits_upgrades_df = labeled_commits_upgrades_df.explode("commit_label")

In [140]:
v1 = labeled_commits_upgrades_df.loc[(labeled_commits_upgrades_df['dependabot_exists']==False)&(labeled_commits_upgrades_df['commit_label']=="Security"), 'delay_hours']
v2 = labeled_commits_upgrades_df.loc[(labeled_commits_upgrades_df['dependabot_exists']==True)&(labeled_commits_upgrades_df['commit_label']=="Security"), 'delay_hours']
v1.median(), v2.median()

(97.00347222222223, 7.510277777777778)

In [147]:
def plot_commit_changes_labels(
    data, x, y, xlab, ylab, filename=None, rotation=0,
    fill=None, fill_lab=None, log_scale=True, colors=None
):
    fig = plt.figure(figsize=(5, 3))

    # Draw main boxplot
    ax = sns.boxplot(
        x=x,
        y=y,
        hue=fill,
        data=data,
        palette=colors,
        patch_artist=True,
        showfliers=False,
        showcaps=False,
        linewidth=0.8
    )

    # ---------------------------------------------------------
    # ADD MEDIAN MARKERS + LABELS
    # ---------------------------------------------------------
    if fill:
        hue_levels = data[fill].unique()
        n_hue = len(hue_levels)
    else:
        hue_levels = [None]
        n_hue = 1

    x_levels = data[x].unique()
    base_positions = np.arange(len(x_levels))

    total_width = 0.8
    each_width = total_width / n_hue

    for xi, xv in enumerate(x_levels):
        for hi, hv in enumerate(hue_levels):
            if fill:
                subset = data[(data[x] == xv) & (data[fill] == hv)]
            else:
                subset = data[data[x] == xv]

            if subset.empty:
                continue

            median_val = subset[y].median()

            # x-position
            if fill:
                xpos = base_positions[xi] - total_width / \
                    2 + each_width/2 + hi * each_width
            else:
                xpos = base_positions[xi]

            # --- Draw the median marker ---
            # ax.scatter(xpos, median_val, marker='D', s=20, color='black', zorder=5)

            # --- Format median value label ---
            if median_val < 24:
                label = f"{median_val:.2f}h"
            elif median_val < 24 * 30:
                label = f"{median_val / 24:.2f}d"
            else:
                label = f"{median_val // (24*30):.2f}mo"

            # --- Draw label next to marker ---
            ax.text(
                xpos,
                median_val,
                label,
                ha='center',
                va='bottom',
                fontsize=6,
                color="black",
                fontweight='bold'
            )

    # ---------------------------------------------------------

    # y-scale + ticks
    if log_scale:
        ax.set_yscale("symlog")

    ax.set_ylim(-0.1)
    hours_yticks = [1, 2, 8, 48, 24*14, 24*60, 24*180]
    ax.set_yticks(hours_yticks)

    def format_tick(h):
        if h < 24:
            return f"{h}h"
        elif h < 24 * 30:
            return f"{h//24}d"
        else:
            return f"{h//(24*30)}mo"

    ax.set_yticklabels([format_tick(h) for h in hours_yticks], fontsize=9)

    # Labels
    plt.xlabel(xlab, fontsize=9)
    plt.ylabel(ylab, fontsize=9)
    plt.xticks(rotation=rotation, ha='right', fontsize=9)

    plt.tight_layout()
    plt.grid(False)

    # Legend
    if fill:
        ax.legend(
            title=fill_lab,
            loc='lower left',
            frameon=True,
            fontsize=8,
            title_fontsize=8,
            bbox_to_anchor=(0.33, 1),
            ncol=len(data[fill].unique())
        )
    else:
        ax.legend([], [], frameon=False)

    if filename:
        # Save the figure
        plt.savefig(filename, bbox_inches='tight', pad_inches=0.01, dpi=300)
        plt.close(fig)
    else:
        plt.show()

In [142]:
label = "Security"
x = 'commit_label'
y = 'delay_hours'
v1, v2 = labeled_commits_upgrades_df[labeled_commits_upgrades_df[x] == label][y], labeled_commits_upgrades_df[labeled_commits_upgrades_df[x] != label][y]
print(cliffs_delta(v1, v2))
print(mannwhitneyu(v1, v2))

(0.05730095701530495, 'negligible')
MannwhitneyuResult(statistic=1576376046.5, pvalue=5.218985991248044e-36)


In [143]:
v1.median(), v2.median()

(14.823888888888888, 9.405)

In [136]:
def compute_cliffs_delta_by_label(df, label_col='labels', group_col='is_security', numeric_cols=None):
    """
    Compute Cliff's delta and p-values (Mann-Whitney U) between security and 
    non-security groups for each label value.
    
    Returns:
        pd.DataFrame : summary with columns ['label', 'feature', 'cliffs_delta', 'effect_size', 'p_value']
    """
    results = []

    if numeric_cols is None:
        numeric_cols = df.select_dtypes(include='number').columns.tolist()
        # Remove group_col from numeric_cols if it happens to be numeric
        numeric_cols = [c for c in numeric_cols if c != group_col]

    for label_val, sub_df in df.groupby(label_col):
        sec_df = sub_df[sub_df[group_col] == "Yes"]
        nonsec_df = sub_df[sub_df[group_col] == "No"]

        if len(sec_df) == 0 or len(nonsec_df) == 0:
            continue 

        for col in numeric_cols:
            v1, v2 = sec_df[col].dropna(), nonsec_df[col].dropna()
            
            # Ensure there is enough data to perform the test
            if len(v1) == 0 or len(v2) == 0:
                continue

            # 1. Compute Cliff's Delta (Effect Size)
            delta, size = cliffs_delta(v1, v2)
            
            # 2. Compute Mann-Whitney U Test (Significance)
            # We use alternative='two-sided' to match standard p-value reporting
            stat, p_val = mannwhitneyu(v1, v2, alternative='two-sided')

            results.append({
                'label': label_val,
                'feature': col,
                'cliffs_delta': delta,
                'effect_size': size,
                'p_value': p_val
            })

    return pd.DataFrame(results).sort_values(['feature', 'cliffs_delta'], ascending=[True, False])

In [137]:
result_df = compute_cliffs_delta_by_label(all_upgrades_df.explode("commit_label"), label_col='commit_label', group_col='is_security', numeric_cols=['delay_time'])
result_df = result_df[result_df['label']!=""]

In [138]:
result_df

,label,feature,cliffs_delta,effect_size,p_value
3,Feature,delay_time,0.391120,medium,0.000000e+00
5,Refactoring,delay_time,0.377520,medium,3.730701e-121
6,Resource,delay_time,0.375044,medium,2.111549e-257
1,Bug,delay_time,0.371736,medium,0.000000e+00
2,Deprecate,delay_time,0.358905,medium,9.752997e-258
8,Test,delay_time,0.344664,medium,4.965312e-220
7,Security,delay_time,0.331750,medium,3.646222e-259
4,Merge,delay_time,0.304811,small,7.561973e-104


##### Plot boxplot figure for types of changes in the commits (each commit label will have to sub-boxplots)

In [8]:
clean_commit_label_upgrades_df = all_upgrades_df.dropna(subset=["commit_label"]).copy()
# clean_commit_label_upgrades_df = clean_commit_label_upgrades_df.explode('commit_label')

In [9]:
# 1. Extract all unique labels
all_labels = sorted(set().union(*clean_commit_label_upgrades_df["commit_label"]))

all_labels = [l for l in all_labels if l not in ['Merge', 'Resource']]

# 2. Build long-format DataFrame for seaborn
rows = []
for label in all_labels:
    yes_mask = clean_commit_label_upgrades_df["commit_label"].apply(lambda x: label in x)

    yes_delays = clean_commit_label_upgrades_df.loc[yes_mask, "delay_hours"]
    repos = clean_commit_label_upgrades_df.loc[yes_mask, "repo"]
    ids = clean_commit_label_upgrades_df.loc[yes_mask, "id"]
    rows.append(pd.DataFrame({
        "label": label,
        "contains": "yes",
        "delay_hours": yes_delays,
        "repo": repos,
        "id": ids,
    }))

    no_delays = clean_commit_label_upgrades_df.loc[~yes_mask, "delay_hours"]
    repos = clean_commit_label_upgrades_df.loc[~yes_mask, "repo"]
    ids = clean_commit_label_upgrades_df.loc[~yes_mask, "id"]
    rows.append(pd.DataFrame({
        "label": label,
        "contains": "no",
        "delay_hours": no_delays,
        "repo": repos,
        "id": ids,
    }))
    
    print(f"Label: {label}, upgrade time (change type included): {yes_delays.median():.2f}, upgrade time (change type not included): {no_delays.median():.2f}")

plot_df = pd.concat(rows, ignore_index=True)

Label: Bug, upgrade time (change type included): 9.09, upgrade time (change type not included): 6.61
Label: Deprecate, upgrade time (change type included): 10.85, upgrade time (change type not included): 7.25
Label: Feature, upgrade time (change type included): 9.03, upgrade time (change type not included): 6.62
Label: Refactoring, upgrade time (change type included): 11.16, upgrade time (change type not included): 7.74
Label: Security, upgrade time (change type included): 14.82, upgrade time (change type not included): 6.75
Label: Test, upgrade time (change type included): 11.57, upgrade time (change type not included): 7.22


In [149]:
plot_commit_changes_labels(
    data=plot_df,
    x="label",
    y="delay_hours",
    xlab="Change type",
    ylab="Upgrade time",
    fill='contains',
    fill_lab="Change type included",
    log_scale=True,
    rotation=45,
    colors=colors,
    filename='figures/RQ5/upgrade_time_per_change_type.pdf'
)

In [10]:
for label in all_labels:
    print(f"====== {label} ======")
    print(mannwhitneyu(
        plot_df.loc[(plot_df['label']==label)&(plot_df['contains']=='yes'), 'delay_hours'],
        plot_df.loc[(plot_df['label']==label)&(plot_df['contains']=='no'), 'delay_hours']
    ))

====== Bug ======
MannwhitneyuResult(statistic=470082550.5, pvalue=1.7725085849080147e-30)
====== Deprecate ======
MannwhitneyuResult(statistic=441593336.5, pvalue=1.5319966718641108e-40)
====== Feature ======
MannwhitneyuResult(statistic=474453870.0, pvalue=4.69163912635631e-35)
====== Refactoring ======
MannwhitneyuResult(statistic=248530478.0, pvalue=1.6133698899987976e-18)
====== Security ======
MannwhitneyuResult(statistic=441570659.0, pvalue=1.4130325834643055e-112)
====== Test ======
MannwhitneyuResult(statistic=412255006.0, pvalue=1.039458447828762e-53)


#### Finding 2: Heatmap of change types against each other

In [156]:
def plot_changetype_heatmap(data_df: pd.DataFrame, filename=None):
    from itertools import combinations
    # 1. Get unique labels
    all_labels = sorted(set().union(*data_df["commit_label"]))

    # Initialize co-occurrence matrix
    cooc = pd.DataFrame(0, index=all_labels, columns=all_labels)

    # 2. Count co-occurrences
    for labels in data_df["commit_label"]:
        labels = list(set(labels))  # ensure no duplicates inside a row
        for (a, b) in combinations(labels, 2):
            cooc.loc[a, b] += 1
            cooc.loc[b, a] += 1
        # Also count self-pair (how many times the label appears alone or with others)
        for a in labels:
            cooc.loc[a, a] += 1

    # 3. Plot heatmap
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.heatmap(
        cooc,
        annot=True,
        fmt="d",
        cmap="Blues",
        linewidths=.2,
        annot_kws={"size": 11}  # smaller font size for cell values
    )

    # Rotate x-axis labels 45 degrees
    plt.xticks(rotation=45, ha="right")

    # Make y-axis labels horizontal
    plt.yticks(rotation=0)

    plt.xlabel("Change type")
    plt.ylabel("Change type")
    plt.tight_layout()

    if filename:
        # Save the figure
        plt.savefig(filename, bbox_inches='tight', pad_inches=0.01, dpi=300)
        plt.close(fig)
    else:
        plt.show()

In [153]:
data_df = clean_commit_label_upgrades_df.copy()
# 2. Remove undesired labels
data_df["commit_label"] = data_df["commit_label"].apply(lambda labels: [x for x in labels if x not in ["Merge", "Resource"]])
data_df = data_df[data_df["commit_label"].map(len) > 0]

In [157]:
plot_changetype_heatmap(
    data_df,
    "figures/RQ5/changetype_heatmap.pdf"
)

#### Finding 3: The upgrade time across version updates

In [169]:
def plot_semantic_vers_upgrade_time_boxplot(data, median_order, median_values, log_scale=True, filename=None):
    # Create figure
    fig, ax = plt.subplots(figsize=(4.5, 2.5))
    
    sns.boxplot(
        data=data,
        x='change_type',
        y='delay_time_hours',
        order=median_order,
        showfliers=False,
        showcaps=False, #
        patch_artist=True,
        hue='change_type',
        # palette='Set3',
        boxprops=dict(facecolor= (0, 0, 0, 0) , edgecolor=(0.5, 0.5, 0.5)),
        palette='dark:none',
        saturation=0.8, linewidth=1.2, ax=ax,
        medianprops={"linewidth": 1, "color": "grey"},
    )
        
    plt.xlabel('Version update')
    plt.ylabel('Upgrade time')

    if log_scale:
        ax.set_yscale("symlog")
    ax.set_ylim(-0.1)
    hours_yticks = [0, 1, 2, 8, 24 * 2, 24 * 14, 24 * 60]
    ax.set_yticks(hours_yticks)
    hours_ylabels = []
    for h in hours_yticks:
        if h < 24:
            hours_ylabels.append(f"{h}h")
        elif h < 24 * 30:
            hours_ylabels.append(f"{h // 24}d")
        else:
            hours_ylabels.append(f"{h // (24 * 30)}mo")
    ax.set_yticklabels(hours_ylabels)

    for i, category in enumerate(median_order):
        median_val = median_values.loc[category]
        y_pos = median_val * 4
        ax.text(i, y_pos, f"{median_val:.2f}h", ha='center', va='top', fontsize=12, color='black')

    ax.grid(False)
    
    fig.tight_layout()

    if filename:
        # Save the figure
        plt.savefig(filename, bbox_inches='tight', pad_inches=0.01, dpi=300)
        plt.close(fig)
    else:
        plt.show()

In [ ]:
# --- 1. Data Conversion ---
# Convert 'delay_time' from minutes to hours, creating a new column for plotting
plot_data_df = all_upgrades_df[all_upgrades_df['change_type'].notna()].copy()
plot_data_df['delay_time_hours'] = plot_data_df['delay_time'] / 60

# --- 2. Sort categories by median delay time and calculate medians ---
median_values = plot_data_df.groupby('change_type')['delay_time_hours'].median().sort_values(ascending=False)
median_order = median_values.index.tolist()

In [171]:
plot_semantic_vers_upgrade_time_boxplot(
    plot_data_df,
    median_order,
    median_values,
    filename="figures/RQ5/upgrade_time_per_version_update.pdf"
)

#### Finding 4: Heatmap of change type and semantic versioning

In [159]:
# Get all unique commit_labels and change_types
def plot_changetype_commit_label_heatmap(df, filename=None):
    df = df[df["change_type"].notna()]
    all_commit_labels = sorted(set().union(*df["commit_label"]))
    all_change_types = sorted(df["change_type"].unique())

    # Initialize co-occurrence matrix
    cooc = pd.DataFrame(0, index=all_commit_labels, columns=all_change_types)

    # Count co-occurrences
    for idx, row in df.iterrows():
        for c_label in row["commit_label"]:
            cooc.loc[c_label, row["change_type"]] += 1

    # Plot heatmap
    fig, ax = plt.subplots(figsize=(4.5, 3.2))
    sns.heatmap(
        cooc,
        annot=True,
        fmt="d",
        cmap="Blues",
        linewidths=.5,
        annot_kws={"size": 10}  # smaller font size for cell values
    )


    plt.xlabel("Semantic version")
    plt.ylabel("Change type")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()

    if filename:
        # Save the figure
        plt.savefig(filename, bbox_inches='tight', pad_inches=0.01, dpi=300)
        plt.close(fig)
    else:
        plt.show()

In [162]:
plot_changetype_commit_label_heatmap(
    data_df,
    "figures/RQ5/sematic_version_heatmap.pdf"
)